In [1]:
from IPython.core.display import HTML, Image, Markdown
import sys
import json
sys.path.append("microTC")
from microtc.textmodel import TextModel
from itertools import combinations


from graphviz import Digraph
dot = Digraph(comment='microtc pipeline', format="png")
# dot.engine = 'circo'
dot.graph_attr['rankdir'] = 'LR'
dot.node('i', '', style="invis")
dot.node('n', 'Normalización')
dot.node('t', 'Tokenización')
dot.node('w', 'Pesado')
dot.node('c', 'Clasificación')
dot.node('o', '', style="invis")

dot.edge('i', 'n', label="texto entrada")
dot.edge('n', 't', label="texto normalizado")
dot.edge('t', 'w', label="bolsa de palabras")
dot.edge('w', 'c', label="vector con pesos")
dot.edge('c', 'o', label="clase")

pipeline = dot.render("fig-pipeline", view=False)


# Clasificación de texto #
## Un enfoque basado en $\mu TC$ ##


**Seminario de la Sociedad Matemática Mexicana SMM 2016**


<div>
    Eric Sadit Téllez Avila INFOTEC 
    <estellezav@conacyt.mx> <br/>
       CONACyT -- INFOTEC
</div>

# Agenda #
- ¿Qué es $\mu TC$
- ¿En qué consiste la tarea de clasificación de texto?
- ¿Cómo esta compuesto $\mu TC$?
- Estado del arte
- Cómo se compara $\mu TC$ con el estado del arte
- Qué falta en $\mu TC$
- Ejemplos


## Categorización de texto ##
El problema consiste en, dado un texto $d$, determinar la(s) categoría(s) a la que pertenece en un conjunto $C$ de categorias, previamente conocido.

Más formalmente:

Dado un conjunto de categorias $\cal{C} = \{c_1, ..., c_m\}$, determinar el subconjunto de categorias
$C_d \in \wp(\cal{C})$ a las que pertenece $d$. 

Notese que $C_t$ puede ser vacio o $\cal{C}$.



## Clasificación de texto ##
La _clasificación_ de texto es una especialización del problema de categorización, donde $|C_d| = 1$, esto es $d$ solo puede ser asignado a una categoría.

Es un problema de interés en la industria y la acádemia, con aplicaciones variadas a distintas áreas del conocimiento.

- Análisis de sentimiento
- Determinación de autoría, e.g., género, edad, estilo, etc.
- Detección de spam
- Categorización de noticias
- Clasificación de idioma

# Procesamiento de Lenguaje Natural #


Un documento $d=s_1\cdots s_n$ es simplemente la concatenación de símbolos $s \in \Sigma$. Donde, $\Sigma$ es un _alfabeto_ de tamaño $\sigma = |\Sigma|$

Notese qué el número de textos posibles de tamaño $n$ es $\sigma^n$, por ejemplo, limitados a texto en inglés en Twitter se tienen
    $$ 26^{140} \simeq 1.248 \times 10^{198} $$

Sin emabargo, en lenguaje natural, este número no suele ser tan grande:
  - existen reglas sobre que símbolos se pueden unir
  - más aún, hay noción de _terminos_ o _palabras_, i.e., _morfología_
  - también, hay reglas sobre como las palabras se pueden combinar, i.e., _sintaxis y gramática_

Sin embargo, es un problema sumamente complicado, hay muchas reglas, variantes, excepciones, errores, etc.

Y por si fuera poco, aunque los conceptos existen en esencia, aparecen de manera diferente en todos los lenguajes


Además, esta el problema semántico:

- un término $s_i$ tiene significados diferentes (antónimos)
- lo contrario también existe, $s_i \not= s_j$ pero que son idénticos en significado (sinónimos)
- en ambos casos, el significado preciso depende del contexto
- también hay casos _aproximados_ de todo lo anterior
- hay muchísimos problemas abiertos

 **NLP** es complicado, de hecho es _AI-complete_


# Nuestro Enfoque #
Por su complejidad, trabajar en NLP tiene una gran cantidad de problemas abiertos, en particular nosotros nos enfocamos en la clasificación de texto escrito de manera informal (e.g., Twitter).

Para esto se utiliza un _pipeline_ estándar

![Pipeline](fig-pipeline.png)

No es la única opción, pero fijar el pipeline es útil como ya se expondrá.


El enfoque teórico suele ser muy complicado, y en realidad poco efectivo en la práctica, dadas las simplificaciones necesarias para hacerlo manejable

- Lógica
- Lingüistica
- Semántica

El enfoque práctico supone muchas cosas, en particular es un tanto casuístico:
    
- Se fija el lenguaje
- Se fija el problema
- Y la raíz de todos los males, muchas veces se supone que entre más técnicas sofísticadas se usen, mejores resultados se tendrán

En ambos enfoques se suele suponer que ausencia de errores de diferentes fuentes, sin embargo, es la regla cuando el texto que se analiza fue generado por usuarios de una red social, e.g. Twitter.

# ¿Qué es $\mu TC$? #
micro TC o $\mu TC$ es un clasificador de texto desarrollado en
el _Laboratorio de Análisis Computacional de Grandes Cúmulos de Información_
(o _Laboratorio de BigDATA_) de INFOTEC, sede Aguascalientes.

Esta disponible para ser clonado en [https://github.com/INGEOTEC/microTC](https://github.com/INGEOTEC/microTC). Esta escrito en Python 3.5 para sacar ventaja de unicode. También se puede instalar utilizando `pip` y `conda`.


En particular, nuestro enfoque se basa en _aprendizaje computacional_ y _optimización combinatoria_. Hemos probado que este esquema es muy competitivo en la práctica. Además, con la adecuada selección de las funciones podemos lograr que 
$\mu TC$ se independiente del lenguaje y robusto a errores.

Esta compuesto por:
- una serie de funciones de transformación de texto
- una serie de tokenizadores
- filtros de palabras y
- algoritmos de pesado de términos

Todo esto orquestado mediante un algoritmo de optimización combinatoria

Entonces, $\mu TC$ optimiza el sub-proceso **normalización $\rightarrow$ tokenizado** para una tarea de clasificación de texto dada. De manera más detallada, una tarea esta definida por $(D, C, f)$

- $D$ es el conjunto de entrenamiento, cada elemento es un documento
- $C$ es el conjunto de clases o etiquetas validas
 * todo $d \in D$ tiene asociado una etiqueda $C_d \in C$
- $f$ es una función de aptitud $f: T \rightarrow \mathbb{R}^+$, i.e., una medida de que tan bien va la clasificación para un conjunto de prueba $T$

El conjunto de prueba $T$ tiene la misma forma de $D$, i.e., $C_d \in C$ para todo ${d \in T}$



## Lista de parametros ##

### Normalizadores multilenguaje ###

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	del-punc | yes, no | Determina si las puntuaciones deben removerse |
|	del-d1   | yes, no | Determina si se deben borrar letras repetidas |
|	del-diac | yes, no | Determina si los simbolos que no ocupan espacios deben ser removidos |
|	lc       | yes, no | Determina si los símbolos deben ser normalizados en minúsculas |
|	emo      | remove, group, none | Controla como deben tratarse los emoticones |
|	num      | remove, group, none | `........................` números |
|	url      | remove, group, none | `........................` urls |
|	usr      | remove, group, none | `........................` usuarios |


### Normalizadores dependientes del lenguaje ###

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	stem   | yes, no | Determina si a las palabras se les aplica _stemming_. |
|	neg    | yes, no | Determina si los operadores de negación son manejados de manera especial |
|	sw | remove, group, none | Controla como los _stopwords_ son manejados |

### Tokenizadores ###
Los tokenizadores son en realidad una lista de tokenizadores, y están definidos tokenizer un elemento en $\wp{(\text{n-words} \cup \text{q-grams} \cup \text{skip-grams})} \setminus \{\emptyset\}$

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	n-words    | $\{1,2,3\}$      | Longitud de n-gramas de palabras (n-words) |
|	q-grams  | $\{1,2,3,4,5,6,7\}$ | Longitud de q-gramas de caracteres) |
|	skip-grams  | $\{(2,1), (3, 1), (2, 2), (3, 2)\}$ | skip-gram list. |

### Parametros para pesado ###
|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|token_min_filter | $\{0.01, 0.03, 0.1, 0.30, -1, -5, -10\}$ | Filtro de frequencias bajas |
|token_max_filter | $\{0.9, 99, 1.0\}$ | Filtro de frequencias altas |
|	tfidf    | yes, no | Determina si se debe realizar un pesado TFIDF de terminos |

In [ ]:
def emoformat(A, emo):
    s = "  ".join([a[0] for a in A if a[1] == emo])
    return s[:1000] + "..."
    
with open('microTC/microtc/resources/emoticons.json') as f:
    A = list(json.load(f).items())
    A.sort()
    S = dict(
        pos=emoformat(A, '_pos'),
        neg=emoformat(A, '_neg'),
        neu=emoformat(A, '_neu'),
        none=emoformat(A, '_none'),
    )

output = ["## Emoticones y emojis clasificados por sentimiento ##"]
for k, v in S.items():
    output.append("## Clase `{0}` ##".format(k.upper()))
    output.append(v)

Markdown("\n".join(output))

In [ ]:

text = "anita lava la tina"
tokenizers = [1, 2, 3, -1, -2, (2,1), (2,2)]

num = 1
output = []
for ltokens in range(len(tokenizers)):
    output.append('## Combinaciones de tamaño {0} ##'.format(ltokens+1))
    output.append('|id|combinación|tokens|')
    output.append('|--|-----------|------|')
    for comb in combinations(tokenizers, ltokens+1):
        model = TextModel([], token_list=comb)
        output.append("|{0}|{1}|{2}|".format(num, comb, ", ".join(model.tokenize(text))))
        num += 1

Markdown("\n".join(output))

## Sobre el pesado ##
El pesado de tokens esta fijo a TFIDF. Su nombre viene de la formulación $tf \times idf$

$tf$ es _term frequency_; es una medida de importancia **local** del término $t$ en el documento $d$, de manera normalizada esta definida como:
    $$tf(t,d) = \frac{freq(t, d)}{\max_{w \in d}{freq(w, d)}}$$
entre más veces aparece en el documento $d$, $t$ es más importante

$idf$ quiere decir _inverse document frequency_; es una medida **global** a la colección $D$, esta definida como:
$$ idf(t,d) = log{\frac{|D|}{1+|{d \in D: t \in d}|}} $$
entre más veces aparece $t$ en la colección, el término es más común y menos discriminante; por lo tanto, menos importante

## Sobre el clasificador ##
El clasificador es un algoritmo de aprendizaje computacional que dado un objeto, decide finalmente la etiqueta o clase de ese objeto. Tiene dos etapas bien definidas

- **Entrenamiento.** Dado un conjunto de ejemplos en un espacio vectorial, con etiquetas, el algoritmo intenta _aprender_ las características que definen cada clase
- **Predicción.** La idea es que una vez entrenado, el algoritmo puede recibir objetos no vistos durante la etapa de entrenamiento y asignales la clase adecuada

En particular, esta fijo como un _Support Vector Machine_ (SVM) con kernel lineal


